In [211]:
import pickle
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.tensorboard import SummaryWriter

In [212]:
class MOSIDataset(Dataset):
    def __init__(self, data_path, split='train'):
        # Load the data from the pickle file
        with open(data_path, 'rb') as f:
            data = pickle.load(f)
        
        # Select the appropriate split
        self.vision = data[split]['vision']
        self.text = data[split]['text']
        self.audio = data[split]['audio']
        self.labels = data[split]['labels']
        self.ids = data[split]['id']

        # audio数据中存在坏点需要处理：
        self.audio[self.audio == float('inf')] = 0.0
        self.audio[self.audio == float('-inf')] = 0.0

    def __len__(self):
        # TODO: 返回数据集长度
        return len(self.labels)

    def __getitem__(self, idx):
        # Extract the features and label for the given index
        vision = torch.tensor(self.vision[idx], dtype=torch.float32)
        text = torch.tensor(self.text[idx], dtype=torch.float32)
        audio = torch.tensor(self.audio[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32).squeeze()

        return vision, text, audio, label

In [213]:
class MultimodalSentimentAnalysisModel(nn.Module):
    def __init__(self):
        super(MultimodalSentimentAnalysisModel, self).__init__()
 
        self.vision_norm = nn.LayerNorm(35)
        self.text_norm = nn.LayerNorm(300)
        self.audio_norm = nn.LayerNorm(74)
        
        self.vision_fc = nn.Linear(35, 128)
        self.text_fc = nn.Linear(300, 128)
        self.audio_fc = nn.Linear(74, 128)
        
        # 定义vision_lstm, text_lstm 和 audio_lstm和融合层mm_lstm. 要求hidden_size=128, num_layers=1, dropout=0.1, batch_first=True
        # TODO: self.vision_lstm
        self.vision_lstm = nn.LSTM(128,128 ,num_layers=1 ,dropout = 0.1, batch_first = True)
        # TODO: self.text_lstm
        self.text_lstm = nn.LSTM(128,128 ,num_layers=1 ,dropout = 0.1, batch_first = True)
        # TODO: self.audio_lstm
        self.audio_lstm = nn.LSTM(128,128 ,num_layers=1 ,dropout = 0.1, batch_first = True)
        # TODO: self.mm_lstm
        self.mm_lstm = nn.LSTM(128,128 ,num_layers=1 ,dropout = 0.1, batch_first = True)
        
        # Define a fully connected layer for fusion
        self.fc = nn.Linear(128, 1)
        
    def forward(self, vision, text, audio):

        # apply layernorm 
        # TODO
        vision = self.vision_norm(vision)
        # TODO
        text = self.text_norm(text)
        # TODO
        audio = self.audio_norm(audio)
        # Process each modality
        vision = F.relu(self.vision_fc(vision))
        text = F.relu(self.text_fc(text))
        audio = F.relu(self.audio_fc(audio))
        
        # LSTM for temporal processing
        output_v, (vision_h, _) = self.vision_lstm(vision)
        output_t, (text_h, _) = self.text_lstm(text)
        output_a, (audio_h, _) = self.audio_lstm(audio)

        # 对单模态的LSTM输出进行直接相加得到feature
        # TODO: feature
        # feature = torch.cat([output_v,output_t, output_a],dim = -1)
        feature = output_v + output_t + output_a
        _, (fusion_tensor, _) = self.mm_lstm(feature)

        # Concatenate the final hidden states
        output = self.fc(fusion_tensor[-1])
        
        # Apply sigmoid to constrain output to (0, 1)
        # TODO
        output = torch.sigmoid(output)
        # Scale and shift to range (-3, 3)
        # TODO
        output = output*6 - 3
        
        return output

In [214]:
def eval_mosi_regression(y_pred, y_true, exclude_zero=False):
    test_preds = y_pred.view(-1).cpu().detach().numpy()
    test_truth = y_true.view(-1).cpu().detach().numpy()

    test_preds_a7 = np.clip(test_preds, a_min=-3., a_max=3.)
    test_truth_a7 = np.clip(test_truth, a_min=-3., a_max=3.)
    test_preds_a5 = np.clip(test_preds, a_min=-2., a_max=2.)
    test_truth_a5 = np.clip(test_truth, a_min=-2., a_max=2.)

    mae = np.mean(np.absolute(test_preds - test_truth))
    corr = np.corrcoef(test_preds, test_truth)[0][1]
    mult_a7 = multiclass_acc(test_preds_a7, test_truth_a7)
    mult_a5 = multiclass_acc(test_preds_a5, test_truth_a5)
    
    non_zeros = np.array([i for i, e in enumerate(test_truth) if e != 0])
    non_zeros_binary_truth = (test_truth[non_zeros] > 0)
    non_zeros_binary_preds = (test_preds[non_zeros] > 0)

    non_zeros_acc2 = accuracy_score(non_zeros_binary_preds, non_zeros_binary_truth)
    non_zeros_f1_score = f1_score(non_zeros_binary_preds, non_zeros_binary_truth, average='weighted')
    
    eval_results = {
        "Non0_acc_2":  round(non_zeros_acc2, 4),
        "Non0_F1_score": round(non_zeros_f1_score, 4),
        "Mult_acc_5": round(mult_a5, 4),
        "Mult_acc_7": round(mult_a7, 4),
        "MAE": round(mae, 4),
        "Corr": round(corr, 4)
    }
    return eval_results

def multiclass_acc(y_pred, y_true):
    y_pred = np.round(y_pred)
    y_true = np.round(y_true)
    acc = 0
    # Compute the accuracy
    # TODO: 注意，这里统计的是总的分类准确率，而不是在每个类别上的准确率。
    acc = (y_pred == y_true).sum().item()
    acc = acc / y_true.shape[0]
    return acc


In [223]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler, device, epochs, writer):
    model.to(device)

    best_corr = 0.
    best_epoch = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for i, (vision, text, audio, labels) in enumerate(train_loader):
            vision, text, audio, labels = vision.to(device), text.to(device), audio.to(device), labels.to(device)
            optimizer.zero_grad()

            # 模型前向获得输出：
            # TODO
            output = model.forward(vision, text, audio)
            # 计算损失：
            # TODO
            loss = criterion(output.squeeze(), labels.squeeze())

            # 反向传播，计算梯度
            # TODO
            loss.backward()
            
            optimizer.step()
            scheduler.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')
        writer.add_scalar('MOSI/train', running_loss/len(train_loader), epoch+1)
        val_corr, val_loss = validate_model(model, valid_loader, criterion, device, writer)
        writer.add_scalar('MOSI/valid', val_loss, epoch+1)
        if val_corr > best_corr:
            best_corr = val_corr
            best_epoch = epoch
            torch.save(model.state_dict(), 'best_model.pth')

    print(f"Best model saved with val_corr {best_corr} at epoch {best_epoch}.")


In [233]:
def validate_model(model, valid_loader, criterion, device, writer):
    model.eval()
    valid_loss = 0.0

    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for vision, text, audio, labels in valid_loader:
            vision, text, audio, labels = vision.to(device), text.to(device), audio.to(device), labels.to(device)
              
            outputs = model(vision, text, audio)
            writer.add_graph(model, [vision, text, audio])
            loss = criterion(outputs.squeeze(), labels.squeeze())

            valid_loss += loss.item()

            all_preds.append(outputs)
            all_labels.append(labels)

    print(f'Validation Loss: {valid_loss/len(valid_loader):.4f}')
    
    all_preds = torch.cat(all_preds, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    # 计算评价指标
    # TODO
    eval_results = eval_mosi_regression(all_preds, all_labels)
    print(eval_results)

    return eval_results["Corr"], valid_loss/len(valid_loader)

In [234]:
def main():
    
    # 固定随机数种子，确保实验结果可重复性
    seed = 42
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(device)
    
    # tensorboard
    writer = SummaryWriter(log_dir='./logs_MOSI/experiment_MOSI')

    # 定义损失函数criterion, 使用均方误差损失。可以使用pytorch封装好的函数，也可以根据公式手写：
    # TODO
    criterion = nn.MSELoss()
    learning_rate = 1e-3
    epochs = 20
    
    # Initialize the model.
    # TODO
    model = MultimodalSentimentAnalysisModel().to(device)
    
    data_path = './mosi_raw.pkl'
    batch_size = 16
    # 初始化训练集和验证集的数据集类
    # TODO: train_dataset
    train_dataset = MOSIDataset(data_path, split="train")
    # TODO: valid_dataset
    valid_dataset = MOSIDataset(data_path, split="valid")
    # 初始化训练集和验证集的加载器，要求batch_size=16
    # TODO: train_loader，参数shuffle=True
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    # TODO: valid_loader，参数shuffle=False
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)
    
    
    # Initialize the optimizer and scheduler.
    # TODO: 使用Adam优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs*len(train_loader))

    # 调用训练函数，注意传入对应参数：
    # TODO
    train_model(model ,train_loader ,valid_loader ,criterion ,optimizer ,scheduler ,device ,epochs, writer)
    # 加载最佳epoch参数
    best_model_state = torch.load('best_model.pth')
    model.load_state_dict(best_model_state)

    # 初始化测试集的数据集类和加载器
    # TODO: test_dataset
    test_dataset = MOSIDataset(data_path, split="test")
    # TODO: test_loader
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
    
    print("\n========== test results: ==========\n")
    validate_model(model, test_loader, criterion, device, writer)
    writer.close()

In [235]:
if __name__ == "__main__":
    main()

cpu


/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Epoch [1/20], Loss: 1.9746
Validation Loss: 1.8236
{'Non0_acc_2': 0.7114, 'Non0_F1_score': 0.7085, 'Mult_acc_5': 0.243, 'Mult_acc_7': 0.2243, 'MAE': np.float32(1.1431), 'Corr': np.float64(0.5961)}
Epoch [2/20], Loss: 1.2605
Validation Loss: 1.5259
{'Non0_acc_2': 0.7313, 'Non0_F1_score': 0.728, 'Mult_acc_5': 0.3037, 'Mult_acc_7': 0.2757, 'MAE': np.float32(1.0512), 'Corr': np.float64(0.6667)}
Epoch [3/20], Loss: 1.0160
Validation Loss: 1.6834
{'Non0_acc_2': 0.7264, 'Non0_F1_score': 0.7258, 'Mult_acc_5': 0.3084, 'Mult_acc_7': 0.2617, 'MAE': np.float32(1.0838), 'Corr': np.float64(0.6666)}
Epoch [4/20], Loss: 0.9591
Validation Loss: 1.5224
{'Non0_acc_2': 0.7512, 'Non0_F1_score': 0.7673, 'Mult_acc_5': 0.3598, 'Mult_acc_7': 0.2944, 'MAE': np.float32(0.9866), 'Corr': np.float64(0.6619)}
Epoch [5/20], Loss: 0.7659
Validation Loss: 1.3909
{'Non0_acc_2': 0.7761, 'Non0_F1_score': 0.7764, 'Mult_acc_5': 0.3598, 'Mult_acc_7': 0.3131, 'MAE': np.float32(0.9834), 'Corr': np.float64(0.6711)}
Epoch [6/20]

/tmp/ipykernel_8136/4143717720.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model_state = torch.load('best_model.pth')


Validation Loss: 1.8300
{'Non0_acc_2': 0.7256, 'Non0_F1_score': 0.7246, 'Mult_acc_5': 0.3207, 'Mult_acc_7': 0.2872, 'MAE': np.float32(1.0784), 'Corr': np.float64(0.5642)}
